In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.neighbors import NearestNeighbors
import numpy as np
import os
import sys
import webbrowser
from time import time_ns

sys.path.append(os.path.join(os.path.dirname(os.getcwd())))
from interfaces.alchemy import AlchemyInterface
import configparser
from db.schema import StationLocation
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
print(os.getcwd())

In [87]:
gdf = gpd.read_file("../shapefiles/world-administrative-boundaries.shp")

In [100]:
coordinates = [[49.6, 49.6], [6, 6]]


def get_country(gdf, latitude, longitude, cache):
    point = Point(longitude, latitude)
    if (latitude, longitude) in cache:
        return cache[(latitude, longitude)]
    possible_matches_index = list(gdf.sindex.intersection(point.bounds))
    if not possible_matches_index:
        return None
    possible_matches = gdf.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(point)]
    result = precise_matches.name.iloc[0] if not precise_matches.empty else None
    cache[(latitude, longitude)] = result
    return result


def get_countries(gdf, coordinates):
    results_cache = {}
    results = []
    for lat, lon in coordinates:
        results.append(get_country(gdf, lat, lon, results_cache))
    return results


get_countries(gdf, coordinates)

['Kazakhstan', 'Nigeria']

In [77]:
coordenadas

,geometry,pais
0,POINT (49.60000 49.60000),Kazakhstan
1,POINT (6.00000 6.00000),Nigeria


,iso3,status,color_code,name,continent,region,iso_3166_1_,french_shor,geometry,geo
0,UGA,Member State,UGA,Uganda,Africa,Eastern Africa,UG,Ouganda,"POLYGON ((33.92110 -1.00194, 33.92027 -1.00111...","POLYGON ((33.92110 -1.00194, 33.92027 -1.00111..."
1,UZB,Member State,UZB,Uzbekistan,Asia,Central Asia,UZ,Ouzbékistan,"POLYGON ((70.97081 42.25467, 70.98054 42.26205...","POLYGON ((70.97081 42.25467, 70.98054 42.26205..."
2,IRL,Member State,IRL,Ireland,Europe,Northern Europe,IE,Irlande,"MULTIPOLYGON (((-9.97014 54.02083, -9.93833 53...","MULTIPOLYGON (((-9.97014 54.02083, -9.93833 53..."
3,ERI,Member State,ERI,Eritrea,Africa,Eastern Africa,ER,Érythrée,"MULTIPOLYGON (((40.13583 15.75250, 40.12861 15...","MULTIPOLYGON (((40.13583 15.75250, 40.12861 15..."
4,None,UK Territory,GBR,South Georgia & the South Sandwich Islands,Antarctica,None,None,South Georgia & the South Sandwich Islands,"MULTIPOLYGON (((-26.24361 -58.49473, -26.24889...","MULTIPOLYGON (((-26.24361 -58.49473, -26.24889..."
...,...,...,...,...,...,...,...,...,...,...
251,BEL,Member State,BEL,Belgium,Europe,Western Europe,BE,Belgique,"POLYGON ((6.01180 50.75727, 6.05472 50.72361, ...","POLYGON ((6.01180 50.75727, 6.05472 50.72361, ..."
252,WSM,Member State,WSM,Samoa,Oceania,Polynesia,WS,Samoa,"MULTIPOLYGON (((-171.42920 -14.01625, -171.441...","MULTIPOLYGON (((-171.42920 -14.01625, -171.441..."
253,AIA,UK Non-Self-Governing Territory,AIA,Anguilla,Americas,Caribbean,AI,Anguilla,"POLYGON ((-63.15375 18.16528, -63.16778 18.164...","POLYGON ((-63.15375 18.16528, -63.16778 18.164..."
254,ISR,Member State,ISR,Israel,Asia,Western Asia,IL,Israël,"POLYGON ((35.62364 33.24573, 35.63249 33.24637...","POLYGON ((35.62364 33.24573, 35.63249 33.24637..."


In [71]:
gdf = gdf.set_index("geo")  # Index by geometry for faster lookup
gdf.set_geometry("geometry")
coordenadas["pais"] = coordenadas.geometry.apply(
    lambda point: gdf[gdf.intersects(point)].iloc[0].iso3
)

IndexError: single positional indexer is out-of-bounds

In [ ]:
gdf

In [ ]:
c_parser = configparser.ConfigParser()
c_parser.read("../../config.ini")
config = {"db": dict(c_parser["db"])}

In [ ]:
alchemy_interface = AlchemyInterface(config)

In [ ]:
time_start = time_ns()
data = alchemy_interface.select_obj(StationLocation)
time_query = time_ns()
print("Time to query data: ", (time_query - time_start) / 1e6, " ms")

In [ ]:
df = pd.DataFrame().from_records(
    [packet for packet in data], columns=StationLocation.__columns__
)

In [ ]:
df.info()

In [ ]:
df = df.dropna(subset=["latitude", "longitude"])

In [ ]:
coords = df[["latitude", "longitude"]].values

# Usar NearestNeighbors para encontrar vecinos en un radio
# El radio está en grados y puedes ajustarlo según tus necesidades
radius = 0.5  # Ajusta este valor según tus necesidades
nbrs = NearestNeighbors(radius=radius, algorithm="ball_tree").fit(coords)
distances, indices = nbrs.radius_neighbors(coords)

# Calcular la densidad: número de puntos en el radio para cada punto
density = np.array([len(indices[i]) for i in range(len(coords))])

# Agregar la densidad al DataFrame original
df["density"] = density

In [ ]:
def do_click(trace, points, state):
    print("clicked")
    if points.point_inds:
        ind = points.point_inds[0]
        url = df.link.iloc[ind]
        webbrowser.open_new_tab(url)


# The logic of drawing a graph
def bar_func(df):
    fig = px.scatter_mapbox(
        df,
        lon="longitude",
        lat="latitude",
    )

    fig.update_traces(
        marker=go.scattermapbox.Marker(
            size=5,
        ),
    )

    fig.update_layout(
        hovermode="closest",
        mapbox_style="dark",
        mapbox_accesstoken=c_parser["mapbox"]["token"],
    )
    fig.data[0].on_click(do_click)
    fig.show()


bar_func(df)

In [ ]:
x = np.random.rand(100)
y = np.random.rand(100)

f = go.FigureWidget([go.Scatter(x=x, y=y, mode="markers")])
scatter = f.data[0]
f.layout.hovermode = "closest"


def do_click(trace, points, state):
    print("Clicked")
    if points.point_inds:
        ind = points.point_inds[0]
        url = df.link.iloc[ind]
        webbrowser.open_new_tab(url)


scatter.on_click(do_click)
f

In [ ]:
import plotly.graph_objects as go

import numpy as np

np.random.seed(1)

x = np.random.rand(100)
y = np.random.rand(100)

f = go.FigureWidget([go.Scatter(x=x, y=y, mode="markers")])

scatter = f.data[0]
colors = ["#a3a7e4"] * 100
scatter.marker.color = colors
scatter.marker.size = [10] * 100
f.layout.hovermode = "closest"


# create our callback function
def update_point(trace, points, selector):
    print("Hemlo")
    # c = list(scatter.marker.color)
    # s = list(scatter.marker.size)
    # for i in points.point_inds:
    #     c[i] = '#bae2be'
    #     s[i] = 20
    #     with f.batch_update():
    #         scatter.marker.color = c
    #         scatter.marker.size = s


scatter.on_click(update_point)

f